In [30]:
#Библиотеки
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import requests
import datetime
import time 
import re 

#Логирование статуса вебсайта
def logize(num): 
    with open("./log.txt",'w') as f:
        pass
        f.write(num)

#Функция ожидания загрузки страницы 15 секунд
def waitForLoad(inputXPath): 
    Wait = WebDriverWait(driver, 15)       
    Wait.until(EC.element_to_be_clickable((By.XPATH, inputXPath)))

#Считывание статуса предыдущей итерации
log = int(pd.read_csv('./log.txt').columns[0]) 
options = webdriver.ChromeOptions()
#options.add_argument('headless')
codec = Service('./yandexdriver.exe')
driver = webdriver.Chrome(service = codec, options=options)

#Подготовка переменных
now = datetime.datetime.now() 
day = now.date().strftime("%d/%m/%Y").replace('/', '.')
hour = now.strftime("%H:%M:%S")
url = 'https://портал-тп.рф/platform/portal/tehprisEE_portal'
text_down = f'Уведомляю Вас, что по состоянию на {day} в {hour} доступ к порталу https://портал-тп.рф прекращен'
text_up = f'Уведомляю Вас, что по состоянию на {day} в {hour} доступ к порталу https://портал-тп.рф возобновлен'
tel = 'https://api.telegram.org/bot5815868306:AAE8cNOYy0_kwVv-hthn0rwvVaMzokZMQj8/sendMessage?chat_id=-888382761'
message_down = {'text': text_down}
message_up = {'text': text_up}
history = []

#Извлечение 
status = requests.head(url) 
code = int(re.search(r"\[([A-Za-z0-9_]+)\]", 
       str(status))[1])

#Проверка доступности страницы в целом
if code == 200: 
    driver.get('https://портал-тп.рф/platform/portal/tehprisEE_portal')

    #Вход в личный кабинет
    private_office = driver.find_element('xpath', '//span[@class="ui-button-text ui-c" and contains(text(), "Личный кабинет")]')
    try:
        private_office.click()
        history.append(1)
    except:
        history.append(0)
    
    if history[0] == 1:

        #Авторизация
        try:
            waitForLoad('//input[@id="workplaceTopForm:j_mail_login"]')
            login_field = driver.find_element('xpath', '//input[@id="workplaceTopForm:j_mail_login"]')
            login_field.send_keys('arselisak@gmail.com')
            password_field = driver.find_element('xpath', '//input[@id="workplaceTopForm:j_password"]')
            password_field.send_keys('SeleniumTest23!')
            login_button = driver.find_element('xpath', '//button[@id="workplaceTopForm:loginBtn"]')
            login_button.click()
            history.append(1)
        except:
            history.append(0)
        
        if history[1] == 1:

            # Элементы личного кабинета
            try:
                waitForLoad('//button[@id="workplaceTopForm:buttonLK"]')
                name_field = driver.find_element('xpath', '//button[@id="workplaceTopForm:buttonLK"]')
                name_field.click()
                private_office = driver.find_element('xpath', '//button[@id="workplaceTopForm:j_idt1545"]')
                private_office.click()
                waitForLoad('//a[contains(text(), "Заявки и обращения")]')
                po_elements = ['//a[contains(text(), "Заявки и обращения")]',
                               '//a[contains(text(), "Опросы")]',
                               '//a[contains(text(), "Договоры")]',
                               '//a[contains(text(), "Уведомления")]',
                               '//a[contains(text(), "История действий")]',
                               '//a[contains(text(), "Подписка")]', 
                               '//a[contains(text(), "Потребление э/э")]']
                po_waits = ['//input[@id="workplaceForm:messagesProfile:myRARProfile:fromDate_input"]',
                            '//a[contains(text(), "Результаты опросов")]',
                            '//div[@class="rpp-scroll"]'
                            '//input[@id="workplaceForm:messagesProfile:selectedFromUsr_input"]',
                            '//input[@id="workplaceForm:messagesProfile:selectedFromAudit_input"]',
                            '//a[@id="workplaceForm:messagesProfile:addSuscribeButton"]',
                            '//a[@class="ui-link ui-widget link-as-btn"]']
                for i in range(len(po_elements)):
                    private_office_el = driver.find_element('xpath', po_elements[i])
                    private_office_el.click()
                    waitForLoad(po_waits[i])
            except:
                history.append(0)


        else:
            for i in range(4):
                history.append(0)
            
    
    else:
        for i in range(5):
            history.append(0)



# 0: Личный ккабинет
# 1: Авторизация
# 2: Элементы личного кабинета
# 3, 4, 5, 6: калькуляторы








        if log == 0: 
            resp = requests.post(tel, data = message_up)
            logize('1')
else:
    if log == 1:
        resp = requests.post(tel, data = message_down)
        logize('0')